In [31]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import mlflow
import os
from sklearn.metrics import f1_score
import pandas as pd
pd.set_option('display.max_columns', None)

import devtalks.helpers.helpers as helpers
wine_dataset_df = helpers.get_wine_dataset()

#### Search Runs

In [74]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
runs = mlflow.search_runs(experiment_ids=['1'])
display(runs)
runs['metrics.val_f1_score']

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_precision_score,metrics.training_recall_score,metrics.training_false_positives,metrics.val_false_positives,metrics.training_score,metrics.test_false_negatives,metrics.val_recall_score,metrics.test_score,metrics.training_example_count,metrics.val_accuracy_score,metrics.val_true_positives,metrics.val_true_negatives,metrics.training_precision_score,metrics.test_accuracy_score,metrics.training_true_negatives,metrics.training_false_negatives,metrics.val_example_count,metrics.test_true_negatives,metrics.training_true_positives,metrics.test_example_count,metrics.training_f1_score,metrics.val_f1_score,metrics.test_false_positives,metrics.test_true_positives,metrics.val_precision_score,metrics.test_recall_score,metrics.val_score,metrics.val_false_negatives,metrics.test_f1_score,metrics.training_accuracy_score,params.average,params.l1_ratio,params.shuffle,params.alpha,params.early_stopping,params.power_t,params.penalty,params.random_state,params.verbose,params.n_iter_no_change,params.class_weight,params.n_jobs,params.tol,params.eta0,params.warm_start,params.loss,params.fit_intercept,params.validation_fraction,params.max_iter,params.epsilon,params.learning_rate,tags.mlflow.pipeline.profile.name,tags.mlflow.log-model.history,tags.mlflow.runName,tags.mlflow.pipeline.step.name,tags.estimator_class,tags.estimator_name,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.pipeline.template.name,tags.mlflow.source.name,tags.mlflow.datasets
0,00fbe67de6ce4f51871a160c9cdcd184,1,FINISHED,file:///C:/Users/honek/Desktop/ML_project_from...,2024-10-14 08:55:36.429000+00:00,2024-10-14 08:55:56.432000+00:00,0.841463,0.905530,106.0,12.0,0.947235,13.0,0.924658,0.936893,4340.0,0.961983,135.0,447.0,0.917510,0.936893,2932.0,123.0,605.0,441.0,1179.0,618.0,0.911480,0.921502,26.0,138.0,0.918367,0.913907,0.961983,11.0,0.876190,0.947235,False,0.15,True,0.0001,False,0.5,l2,42,0,5,"{0: 0.6638972809667674, 1: 2.025345622119816}",None,0.001,0.0,False,hinge,True,0.1,1000,0.1,optimal,local,"[{""run_id"": ""00fbe67de6ce4f51871a160c9cdcd184""...",zealous-wolf-475,register,sklearn.linear_model._stochastic_gradient.SGDC...,SGDClassifier,honek,RECIPE,classification/v1,C:\Users\honek\AppData\Local\miniconda3\envs\d...,"[{""name"":""69efffe57c2aa29db29695c55788a2df"",""h..."
1,8372d889be244ceeb34728edfbd78fea,1,FINISHED,file:///C:/Users/honek/Desktop/ML_project_from...,2024-10-13 17:19:10.999000+00:00,2024-10-13 17:19:32.317000+00:00,0.600000,0.980799,516.0,91.0,0.875346,4.0,0.986301,0.834951,4340.0,0.846281,144.0,368.0,0.712214,0.834951,2522.0,25.0,605.0,369.0,1277.0,618.0,0.825202,0.755906,98.0,147.0,0.612766,0.973510,0.846281,2.0,0.742424,0.875346,False,0.15,True,0.0001,False,0.5,l2,42,0,5,"{0: 0.6638972809667674, 1: 2.025345622119816}",None,0.001,0.0,False,hinge,True,0.1,1000,0.1,optimal,local,"[{""run_id"": ""8372d889be244ceeb34728edfbd78fea""...",stately-pug-909,register,sklearn.linear_model._stochastic_gradient.SGDC...,SGDClassifier,honek,RECIPE,classification/v1,C:\Users\honek\AppData\Local\miniconda3\envs\d...,"[{""name"":""a3d0d4a44a98a66988f0e717964659f9"",""h..."
2,91f56081409c4161bf8651d9b135cb74,1,FINISHED,file:///C:/Users/honek/Desktop/ML_project_from...,2024-10-13 15:08:28.374000+00:00,2024-10-13 15:08:50.759000+00:00,0.936508,0.814900,47.0,6.0,0.933641,33.0,0.835616,0.933657,4340.0,0.950413,122.0,453.0,0.957581,0.933657,2991.0,241.0,605.0,459.0,1061.0,618.0,0.880498,0.890511,8.0,118.0,0.953125,0.781457,0.950413,24.0,0.851986,0.933641,False,0.15,True,0.0001,False,0.5,l2,42,0,5,"{0: 0.6638972809667674, 1: 2.025345622119816}",None,0.001,0.0,False,hinge,True,0.1,1000,0.1,optimal,local,"[{""run_id"": ""91f56081409c4161bf8651d9b135cb74""...",monumental-mole-869,train,sklearn.linear_model._stochastic_gradient.SGDC...,SGDClassifier,honek,RECIPE,classification/v1,C:\Users\honek\AppData\Local\miniconda3\envs\d...,"[{""name"":""bfe564fbb8d2e9e895378ba44ff2a08e"",""h..."
3,134ce3bb2b6b4c689feaf7bc

0    0.921502
1    0.755906
2    0.890511
3    0.819767
4    0.896797
5    0.705596
6    0.865204
Name: metrics.val_f1_score, dtype: float64

#### Get Ordered Runs

In [75]:
ordered_runs = mlflow.search_runs(experiment_ids=['1'], order_by=["metrics.val_f1_score DESC"])
display(ordered_runs)
ordered_runs['metrics.val_f1_score']

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_precision_score,metrics.training_recall_score,metrics.training_false_positives,metrics.val_false_positives,metrics.training_score,metrics.test_false_negatives,metrics.val_recall_score,metrics.test_score,metrics.training_example_count,metrics.val_accuracy_score,metrics.val_true_positives,metrics.val_true_negatives,metrics.training_precision_score,metrics.test_accuracy_score,metrics.training_true_negatives,metrics.training_false_negatives,metrics.val_example_count,metrics.test_true_negatives,metrics.training_true_positives,metrics.test_example_count,metrics.training_f1_score,metrics.val_f1_score,metrics.test_false_positives,metrics.test_true_positives,metrics.val_precision_score,metrics.test_recall_score,metrics.val_score,metrics.val_false_negatives,metrics.test_f1_score,metrics.training_accuracy_score,params.average,params.l1_ratio,params.shuffle,params.alpha,params.early_stopping,params.power_t,params.penalty,params.random_state,params.verbose,params.n_iter_no_change,params.class_weight,params.n_jobs,params.tol,params.eta0,params.warm_start,params.loss,params.fit_intercept,params.validation_fraction,params.max_iter,params.epsilon,params.learning_rate,tags.mlflow.pipeline.profile.name,tags.mlflow.log-model.history,tags.mlflow.runName,tags.mlflow.pipeline.step.name,tags.estimator_class,tags.estimator_name,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.pipeline.template.name,tags.mlflow.source.name,tags.mlflow.datasets
0,00fbe67de6ce4f51871a160c9cdcd184,1,FINISHED,file:///C:/Users/honek/Desktop/ML_project_from...,2024-10-14 08:55:36.429000+00:00,2024-10-14 08:55:56.432000+00:00,0.841463,0.905530,106.0,12.0,0.947235,13.0,0.924658,0.936893,4340.0,0.961983,135.0,447.0,0.917510,0.936893,2932.0,123.0,605.0,441.0,1179.0,618.0,0.911480,0.921502,26.0,138.0,0.918367,0.913907,0.961983,11.0,0.876190,0.947235,False,0.15,True,0.0001,False,0.5,l2,42,0,5,"{0: 0.6638972809667674, 1: 2.025345622119816}",None,0.001,0.0,False,hinge,True,0.1,1000,0.1,optimal,local,"[{""run_id"": ""00fbe67de6ce4f51871a160c9cdcd184""...",zealous-wolf-475,register,sklearn.linear_model._stochastic_gradient.SGDC...,SGDClassifier,honek,RECIPE,classification/v1,C:\Users\honek\AppData\Local\miniconda3\envs\d...,"[{""name"":""69efffe57c2aa29db29695c55788a2df"",""h..."
1,151848609c0d4e2d9e76c923009a019f,1,FINISHED,file:///C:/Users/honek/Desktop/ML_project_from...,2024-10-11 12:44:40.740000+00:00,2024-10-11 12:45:02.273000+00:00,0.903448,0.860983,64.0,9.0,0.943548,20.0,0.863014,0.944984,4340.0,0.952066,126.0,450.0,0.945992,0.944984,2974.0,181.0,605.0,453.0,1121.0,618.0,0.901488,0.896797,14.0,131.0,0.933333,0.867550,0.952066,20.0,0.885135,0.943548,False,0.15,True,0.0001,False,0.5,l2,42,0,5,"{0: 0.6638972809667674, 1: 2.025345622119816}",None,0.001,0.0,False,hinge,True,0.1,1000,0.1,optimal,local,"[{""run_id"": ""151848609c0d4e2d9e76c923009a019f""...",silent-worm-79,train,sklearn.linear_model._stochastic_gradient.SGDC...,SGDClassifier,honek,RECIPE,classification/v1,C:\Users\honek\AppData\Local\miniconda3\envs\d...,"[{""name"":""471005db01b1b1c76d005c13c0845fc0"",""h..."
2,91f56081409c4161bf8651d9b135cb74,1,FINISHED,file:///C:/Users/honek/Desktop/ML_project_from...,2024-10-13 15:08:28.374000+00:00,2024-10-13 15:08:50.759000+00:00,0.936508,0.814900,47.0,6.0,0.933641,33.0,0.835616,0.933657,4340.0,0.950413,122.0,453.0,0.957581,0.933657,2991.0,241.0,605.0,459.0,1061.0,618.0,0.880498,0.890511,8.0,118.0,0.953125,0.781457,0.950413,24.0,0.851986,0.933641,False,0.15,True,0.0001,False,0.5,l2,42,0,5,"{0: 0.6638972809667674, 1: 2.025345622119816}",None,0.001,0.0,False,hinge,True,0.1,1000,0.1,optimal,local,"[{""run_id"": ""91f56081409c4161bf8651d9b135cb74""...",monumental-mole-869,train,sklearn.linear_model._stochastic_gradient.SGDC...,SGDClassifier,honek,RECIPE,classification/v1,C:\Users\honek\AppData\Local\miniconda3\envs\d...,"[{""name"":""bfe564fbb8d2e9e895378ba44ff2a08e"",""h..."
3,2d041ce658d44438958b3d2150a

0    0.921502
1    0.896797
2    0.890511
3    0.865204
4    0.819767
5    0.755906
6    0.705596
Name: metrics.val_f1_score, dtype: float64

#### Make Inference Using Selected Model

In [83]:
which_run = 0
model_uri = f"runs:/{ordered_runs.iloc[which_run, :].run_id}/train/model"
print(model_uri)
model = mlflow.pyfunc.load_model(model_uri)
y_true = wine_dataset_df['is_red']
y_pred = model.predict(wine_dataset_df)
print(f'f1_score: {f1_score(y_true, y_pred)}')

runs:/00fbe67de6ce4f51871a160c9cdcd184/train/model
f1_score: 0.9024238657551275
